In [ ]:
import numpy as np
import pandas as pd
import re
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import SGDClassifier
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import f1_score
from sklearn.model_selection import GridSearchCV
from lightgbm import LGBMClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import StackingClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB

<h2><strong><font color ='blue'>Data Cleaning & Preprocessing</font></strong></h2>

In [ ]:
train_data=pd.read_excel('train_Data.xlsx')
train_bureau=pd.read_excel('train_bureau.xlsx')
print(train_data.shape)
print(train_bureau.shape)

(128655, 26)
(560844, 25)


In [ ]:
test_data=pd.read_excel('test_Data.xlsx')
test_bureau=pd.read_excel('test_bureau.xlsx')
print(test_data.shape)
print(test_bureau.shape)

(14745, 25)
(64019, 25)


In [ ]:
print('columns with null values')
for column in train_data.columns:
  if train_data[column].isnull().values.any():
    print(column)

columns with null values
Area
MaturityDAte
SEX
AGE
MonthlyIncome
City
ZiPCODE


In [ ]:
print('Mean value of MonthlyIncome : ',train_data['MonthlyIncome'].mean())
print('Mean value of AGE : ',train_data['AGE'].mean())

Mean value of MonthlyIncome :  50323.60434088012
Mean value of AGE :  40.66480294876979


In [ ]:
train_data['SEX'].value_counts()

M    122144
F      6452
Name: SEX, dtype: int64

In [ ]:
train_data['ZiPCODE'].value_counts()

125001.0    806
125055.0    690
584128.0    524
125050.0    409
334001.0    407
           ... 
583234.0      1
583233.0      1
583215.0      1
244104.0      1
524299.0      1
Name: ZiPCODE, Length: 9123, dtype: int64

In [ ]:
def filling_nan(data):
  data['Area']=data['Area'].fillna('UNKNOWN')
  data['SEX']=data['SEX'].fillna('M')
  data['AGE']=data['AGE'].fillna(40.0)
  data['MonthlyIncome']=data['MonthlyIncome'].fillna(50323.6)
  data['City']=data['City'].fillna('UNKNOWN')
  data['ZiPCODE']=data['ZiPCODE'].fillna(125001.0)
  data['ZiPCODE']=data['ZiPCODE']/10000
  return data

In [ ]:
train_data=filling_nan(train_data)
train_data.dropna(inplace=True)
train_data.shape
test_data=filling_nan(test_data)
test_data.fillna(0,inplace=True)

In [ ]:
category_columns=['Frequency','InstlmentMode','LoanStatus','PaymentMode','Area','SEX','City','State']
labels_dict={}
for col in category_columns:
  unique_cat=list(train_data[col].unique())
  dicts=dict((value, index) for index,value in enumerate(unique_cat))
  labels_dict[col]=dicts


def transform_categorical_features(columns,data):
  for column in columns:
    label_dict=labels_dict[column]
    data[column]=data[column].apply(lambda x: label_dict[x] if x in label_dict.keys() else -1)
  return data

train_data=transform_categorical_features(category_columns,train_data)
test_data=transform_categorical_features(category_columns,test_data)

<h2><strong><font color ='blue'>Feature Extraction</font></strong></h2>

In [ ]:
train_data['no_of_loans']=train_data['ID'].apply(lambda id: len(train_bureau[train_bureau['ID']==id]) )
test_data['no_of_loans']=test_data['ID'].apply(lambda id: len(test_bureau[train_bureau['ID']==id]) )

<h2>Defining Functions needed for feature extraction </h2>

In [ ]:
def get_value_counts(id,data,column,categories):
  '''It takes id and column to return all the category counts in that column'''
  counts=data[data['ID']==id][column].value_counts()
  values=[]
  for cat in categories:
    if cat in counts.keys():
      values.append(counts[cat])
    else:
      values.append(0)
  return values

In [ ]:
def get_values(hist_data):
  '''It takes reported date histogram values and calculates difference between each date and does median'''
  if type(hist_data)!=str:
    return '0'
  dates=hist_data.split(',')
  dates=[date for date in dates if len(date)==8]
  diff_days=[]
  if len(dates)>=2:
    for i in range(len(dates)-1):
      diff=pd.to_datetime(dates[i], format='%Y%m%d', errors='ignore')-pd.to_datetime(dates[i+1], format='%Y%m%d', errors='ignore')
      diff_days.append(diff.days)
  if len(dates)==1:
    diff_days.append(0)
  if len(dates)!=0:
    return [np.mean(diff_days),np.std(diff_days),np.median(diff_days)]
  else:
    return '0'

In [ ]:
def get_reported_hist_feat(id,data):
  '''It extracts features from reported histogram data'''
  id_data=data[data['ID']==id]
  values=[]
  for index,row in id_data.iterrows():
    hist_data=row['REPORTED DATE - HIST']
    values.append(get_values(hist_data))
  values=[value for value in values if value!='0']

  return np.median(values,axis=0)

In [ ]:
def get_amt_values(hist_data):
  '''It extracts the values from ammount histogram data'''
  if type(hist_data)!=str:
    return '0'
  values=hist_data
  values=values.split(',')
  values=[int(value.split('.')[0]) for value in values if len(value.split('.')[0])!=0]
  if len(values)!=0:
    return [np.mean(values),np.std(values),np.median(values)]
  else:
    return '0'

In [ ]:
def get_amt_hist_feat(id,data,column):
  '''It return features extracted from ammount histogram data'''
  id_data=data[data['ID']==id]
  print(id)
  values=[]
  for index,row in id_data.iterrows():
    hist_data=row[column]
    values.append(get_amt_values(hist_data))
  values=[value for value in values if value!='0']
  if len(values)!=0:
    return np.median(values,axis=0)
  else:
    return [0,0,0]

In [ ]:
def get_cross_tabs(data,bureau,columns_crosstab):
  '''It performs crosstab operation on mentioned columns'''
  for column in columns_crosstab:
    data[column+' COUNTS']=data['ID'].apply(lambda id : get_value_counts(id,bureau,column,categories_dict[column]))
    for index,category in enumerate(categories_dict[column]):
      data[column+'_'+category]=data[column+' COUNTS'].apply(lambda x: x[index])

In [ ]:
def get_seq(hist_data):
  '''In DPD histogram it converts exponential values to string formated number seq'''
  num_e=hist_data
  pow=num_e.split('+')[1]
  pre=num_e.split('E')[0].replace('.','')
  seq=pre+''.join(['0' for i in range(int(pow))])

  return seq

In [ ]:
def get_dpd_feat(hist_data):
  '''This takes DPD histograms and catgeorises id into 7 categories'''
  grade_A,grade_B,grade_C,grade_D,grade_E,grade_F=0,0,0,0,0,0
  if type(hist_data)!=str:
    return [grade_A,grade_B,grade_C,grade_D,grade_E,grade_F]
  if 'E' in hist_data:
    hist_data=get_seq(hist_data)
  delay_days=re.findall('...',hist_data)
  for delay in delay_days:
    if delay=='DDD' or delay=='XXX':
      continue
    if int(delay)==0:
      grade_A+=1
    if int(delay)>0 and int(delay)<100:
      grade_B+=1
    if int(delay)>=100 and int(delay)<500:
      grade_C+=1
    if int(delay)>=500 and int(delay)<900:
      grade_D+=1
    if int(delay)>=900 and int(delay)<999:
      grade_E+=1
    if int(delay)==999 :
      grade_F+=1
  return [grade_A,grade_B,grade_C,grade_D,grade_E,grade_F]

In [ ]:
def extract_dpd_feat(data,bureau):
  '''EXtracts DPD features'''
  bureau['dpd_feat']=bureau['DPD - HIST'].apply(lambda hist_data: get_dpd_feat(hist_data))
  data['dpd_feat']=data['ID'].apply(lambda id : np.sum(list(bureau[bureau['ID']==id]['dpd_feat'].values),axis=0))
  
  data['dpd_gradeA']=data['dpd_feat'].apply(lambda x : x[0])
  data['dpd_gradeB']=data['dpd_feat'].apply(lambda x : x[1])
  data['dpd_gradeC']=data['dpd_feat'].apply(lambda x : x[2])
  data['dpd_gradeD']=data['dpd_feat'].apply(lambda x : x[3])
  data['dpd_gradeE']=data['dpd_feat'].apply(lambda x : x[4])
  data['dpd_gradeF']=data['dpd_feat'].apply(lambda x : x[5])

In [ ]:
def get_asset_class_feat(data,bureau):
  '''Extracts asset-class features'''
  categories=['Standard','SubStandard','Special Mention Account','Doubtful','Loss','1']
  for category in categories:
    data['Is Asset class '+category]=data['ID'].apply(lambda id: 1 if category in list(bureau[bureau['ID']==id]['ASSET_CLASS'].values) else 0)

In [ ]:
def extract_features(data,bureau):

 data['self_indicator_values']=data['ID'].apply(lambda id: get_value_counts(id,bureau,'SELF-INDICATOR',[True,False]))
 data['self_indicator_true']=data['self_indicator_values'].apply(lambda x: x[0])
 data['self_indicator_false']=data['self_indicator_values'].apply(lambda x: x[1])


 data['self_indicator_false']=data['self_indicator_false']/data['no_of_loans']
 data['self_indicator_true']=data['self_indicator_true']/data['no_of_loans']


 data['total_disbursal_amt(in lakhs)']=data['ID'].apply(lambda id: bureau[bureau['ID']==id]['DISBURSED-AMT/HIGH CREDIT'].sum()/100000)
 data['avg_disbursal_amt']=data['total_disbursal_amt(in lakhs)']/data['no_of_loans']

 data['total_cur_bal(in lakhs)']=data['ID'].apply(lambda id: bureau[bureau['ID']==id]['CURRENT-BAL'].sum()/100000)
 data['avg_cur_bal']=data['total_cur_bal(in lakhs)']/data['no_of_loans']


 data['reported_hist_feat']=data['ID'].apply(lambda id: get_reported_hist_feat(id,bureau))
 data['reported_hist_median_mean']=data['reported_hist_feat'].apply(lambda x : x[0])
 data['reported_hist_median_std']=data['reported_hist_feat'].apply(lambda x : x[1])
 data['reported_hist_median_median']=data['reported_hist_feat'].apply(lambda x : x[2])


 data['cur_bal_hist_feat']=data['ID'].apply(lambda id: get_amt_hist_feat(id,bureau,'CUR BAL - HIST'))
 data['cur_bal_hist_median_mean']=data['cur_bal_hist_feat'].apply(lambda x : x[0])
 data['cur_bal_hist_median_std']=data['cur_bal_hist_feat'].apply(lambda x : x[1] )
 data['cur_bal_hist_median_median']=data['cur_bal_hist_feat'].apply(lambda x : x[2] )

 data['amt_overdue_hist_feat']=data['ID'].apply(lambda id: get_amt_hist_feat(id,bureau,'AMT OVERDUE - HIST'))
 data['amt_overdue_hist_median_mean']=data['amt_overdue_hist_feat'].apply(lambda x : x[0])
 data['amt_overdue__hist_median_std']=data['amt_overdue_hist_feat'].apply(lambda x : x[1] )
 data['amt_overdue__hist_median_median']=data['amt_overdue_hist_feat'].apply(lambda x : x[2] )

 data['amt_paid_hist_feat']=data['ID'].apply(lambda id: get_amt_hist_feat(id,bureau,'AMT PAID - HIST'))
 data['amt_paid_hist_median_mean']=data['amt_paid_hist_feat'].apply(lambda x : x[0])
 data['amt_paid__hist_median_std']=data['amt_paid_hist_feat'].apply(lambda x : x[1] )
 data['amt_paid__hist_median_median']=data['amt_paid_hist_feat'].apply(lambda x : x[2] )


 data['cur_bal_hist_median_mean']=data['cur_bal_hist_median_mean']/100000
 data['cur_bal_hist_median_std']=data['cur_bal_hist_median_std']/100000
 data['cur_bal_hist_median_median']=data['cur_bal_hist_median_median']/100000
 data['amt_paid_hist_median_mean']=data['amt_paid_hist_median_mean']/100000
 data['amt_paid__hist_median_std']=data['amt_paid__hist_median_std']/100000
 data['amt_paid__hist_median_median']=data['amt_paid__hist_median_median']/100000
 data['amt_overdue_hist_median_mean']=data['amt_overdue_hist_median_mean']/100000
 data['amt_overdue__hist_median_std']=data['amt_overdue__hist_median_std']/100000
 data['amt_overdue__hist_median_median']=data['amt_overdue__hist_median_median']/100000

In [ ]:
categories_dict={}
columns_crosstab=['OWNERSHIP-IND','CONTRIBUTOR-TYPE','ACCOUNT-STATUS','ACCT-TYPE']
for column in columns_crosstab:
  categories_dict[column]=list(train_bureau[column].unique())

get_cross_tabs(train_data,train_bureau)
get_cross_tabs(test_data,test_bureau)

extract_dpd_feat(train_data,train_bureau)
extract_dpd_feat(test_data,test_bureau)
  
get_asset_class_feat(train_data,train_bureau)
get_asset_class_feat(test_data,test_bureau)

In [ ]:
extract_features(train_data,train_bureau)
extract_features(test_data,test_bureau)

In [ ]:
to_remove_features=['AssetID','self_indicator_values','reported_hist_feat','cur_bal_hist_feat','amt_overdue_hist_feat','dpd_feat',
                                     'amt_paid_hist_feat','OWNERSHIP-IND COUNTS','CONTRIBUTOR-TYPE COUNTS','ACCOUNT-STATUS COUNTS','ACCT-TYPE COUNTS']

#Removing unnecessary columns

train_data.drop(to_remove_features,axis=1,inplace=True)
test_data.drop(to_remove_features,axis=1,inplace=True)

In [ ]:
train_data.head()

,ID,Frequency,InstlmentMode,LoanStatus,PaymentMode,BranchID,Area,Tenure,AssetCost,AmountFinance,DisbursalAmount,EMI,ManufacturerID,SupplierID,LTV,SEX,AGE,MonthlyIncome,City,State,ZiPCODE,no_of_loans,self_indicator_true,self_indicator_false,avg_disbursal_amt,total_disbursal_amt(in lakhs),total_cur_bal(in lakhs),avg_cur_bal,reported_hist_median_mean,reported_hist_median_std,reported_hist_median_median,cur_bal_hist_median_mean,cur_bal_hist_median_std,cur_bal_hist_median_median,amt_overdue_hist_median_mean,amt_overdue__hist_median_std,amt_overdue__hist_median_median,amt_paid_hist_median_mean,amt_paid__hist_median_std,amt_paid__hist_median_median,...,ACCT-TYPE_Microfinance Housing Loan,ACCT-TYPE_Loan on Credit Card,ACCT-TYPE_Microfinance Personal Loan,ACCT-TYPE_Telco Landline,ACCT-TYPE_SHG Group,ACCT-TYPE_Business Non-Funded Credit Facility-Priority Sector-Others,dpd_gradeA,dpd_gradeB,dpd_gradeC,dpd_gradeD,dpd_gradeE,dpd_gradeF,DisbursalDate_year,DisbursalDate_month,DisbursalDate_week,DisbursalDate_day,DisbursalDate_hour,DisbursalDate_minute,DisbursalDate_dayofweek,MaturityDAte_year,MaturityDAte_month,MaturityDAte_week,MaturityDAte_day,MaturityDAte_hour,MaturityDAte_minute,MaturityDAte_dayofweek,AuthDate_year,AuthDate_month,AuthDate_week,AuthDate_day,AuthDate_hour,AuthDate_minute,AuthDate_dayofweek,Is Asset class Standard,Is Asset class SubStandard,Is Asset class Special Mention Account,Is Asset class Doubtful,Is Asset class Loss,Is Asset class 1,Top-up Month
0,1,0,0,0,0,1,0,48,4.50000,2.75000,2.75000,0.240,1568,21946,61.11,0,49.0,0.358333,0,0,46.4993,9,0.555556,0.444444,2.445947,22.01352,6.18526,0.687251,30.428571,0.842650,31.0,1.256689,0.180733,1.251155,0.009621,0.0,0.01019,0.000000,0.000000,0.00000,...,0,0,0,0,0,0,113,75,0,0,0,0,2012,2,6,10,0,0,4,2016,1,2,15,0,0,4,2012,2,6,10,0,0,4,1,0,0,0,0,0,> 48 Months
1,2,0,1,0,1,333,1,47,4.85000,3.50000,3.50000,0.105,1062,34802,70.00,0,23.0,0.006667,1,0,46.6001,13,0.076923,0.923077,13.936216,181.17081,103.49457,7.961121,30.416667,0.842650,31.0,4.658945,1.286095,4.364365,0.000000,0.0,0.00000,0.385600,0.000022,0.20315,...,0,0,0,0,0,0,179,31,0,0,0,0,2012,3,13,31,0,0,5,2016,2,7,15,0,0,0,2012,3,13,31,0,0,5,1,0,0,0,0,0,No Top-up Service
2,3,1,0,1,2,1,0,68,6.90000,5.19728,5.19728,0.383,1060,127335,69.77,0,39.0,0.452570,2,0,46.2030,31,0.064516,0.935484,1.196248,37.08369,20.65165,0.666182,30.421053,0.815365,31.0,0.406565,0.110529,0.462500,0.000000,0.0,0.00000,0.052318,0.030938,0.01894,...,0,0,0,0,0,0,118,32,3,2,1,0,2017,6,24,17,0,0,5,2023,2,6,10,0,0,4,2017,6,24,17,0,0,5,1,0,0,0,0,0,12-18 Months
3,7,0,1,0,3,125,2,48,4.80000,4.00000,4.00000,0.116,1060,25094,80.92,0,24.0,0.208333,3,0,47.3335,4,0.500000,0.500000,3.181382,12.72553,6.24000,1.560000,30.464286,0.813405,31.0,1.748441,0.743997,1.952077,0.000000,0.0,0.00000,0.000000,0.000000,0.00000,...,0,0,0,0,0,0,79,0,0,0,0,0,2013,11,48,29,0,0,4,2017,11,45,10,0,0,4,2013,11,48,29,0,0,4,0,0,0,0,0,0,> 48 Months
4,8,0,0,0,3,152,3,44,6.19265,4.40000,4.40000,0.150,1046,21853,71.05,0,56.0,0.273137,4,1,49.5442,7,0.142857,0.857143,2.893166,20.25216,9.74119,1.391599,30.416667,0.868496,31.0,1.178993,0.429246,1.721090,0.000000,0.0,0.00000,0.197883,0.034589,0.17760,...,0,0,0,0,0,0,86,15,4,0,0,0,2011,12,49,8,0,0,3,2015,7,27,5,0,0,6,2011,12,49,8,0,0,3,1,0,0,0,0,0,36-48 Months


<h2><strong><font color ='blue'>Modeling</font></strong></h2>

In [ ]:
X_train=train_data.drop(['ID','Top-up Month'],axis=1)
y_train=pd.DataFrame(train_data['Top-up Month'],columns=['Top-up Month'])
X_test=test_data.drop(['ID'],axis=1)
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)

(128654, 150)
(128654, 1)
(14745, 150)


In [ ]:
ohe_target=OneHotEncoder()
y_train=ohe_target.fit_transform(y_train)
y_train=y_train.toarray()
y_train=pd.DataFrame(y_train,columns=ohe_target.categories_)
y_train.head(2)

,> 48 Months,12-18 Months,18-24 Months,24-30 Months,30-36 Months,36-48 Months,No Top-up Service
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [ ]:
X_train, X_cv, y_train, y_cv = train_test_split(X_train, y_train,stratify=y_train, test_size=0.3, random_state=42)

In [ ]:
scalar=StandardScaler()
X_standardized_train=scalar.fit_transform(X_train)
X_standardized_cv=scalar.transform(X_cv)
X_standardized_test=scalar.transform(X_test)

X_train=pd.DataFrame(X_standardized_train,columns=X_train.columns)
X_cv=pd.DataFrame(X_standardized_cv,columns=X_cv.columns)
X_test=pd.DataFrame(X_standardized_test,columns=X_test.columns)

In [ ]:
labels=[' > 48 Months','12-18 Months','18-24 Months','24-30 Months','30-36 Months','36-48 Months','No Top-up Service']

In [ ]:
def train_base_model(cat,depth=12,estimators=150):
  clf=RandomForestClassifier(max_depth=depth,n_estimators=estimators,class_weight='balanced',n_jobs=-1,random_state=42)
  clf.fit(X_train,np.ravel(y_train[cat]))

  y_pred=clf.predict(X_train)
  y_pred_train[cat[0]]=y_pred
  F1_score=f1_score(np.ravel(y_train[cat]),y_pred,average='macro')
  print('Train '+cat[0]+' F1-Score : ',F1_score)
  
  y_pred=clf.predict(X_cv)
  y_pred_cv[cat[0]]=y_pred
  F1_score=f1_score(np.ravel(y_cv[cat]),y_pred,average='macro')
  print('CV '+cat[0]+' F1-Score : ',F1_score)

  y_pred=clf.predict(X_test)
  y_pred_test[cat[0]]=y_pred

  return clf

In [ ]:
baseline_models=[]
y_pred_train=pd.DataFrame()
y_pred_cv=pd.DataFrame()
y_pred_test=pd.DataFrame()
for index,value in enumerate(labels):
  clf=train_base_model([value],11,150)
  baseline_models.append(clf)

Train  > 48 Months F1-Score :  0.69680237023857
CV  > 48 Months F1-Score :  0.6575298585555662
Train 12-18 Months F1-Score :  0.692193627126898
CV 12-18 Months F1-Score :  0.5899303293904434
Train 18-24 Months F1-Score :  0.6680298949738751
CV 18-24 Months F1-Score :  0.610751793924319
Train 24-30 Months F1-Score :  0.670774428642185
CV 24-30 Months F1-Score :  0.6176741982678606
Train 30-36 Months F1-Score :  0.6050971192531808
CV 30-36 Months F1-Score :  0.5661176656089746
Train 36-48 Months F1-Score :  0.5966419073152457
CV 36-48 Months F1-Score :  0.553843088567934
Train No Top-up Service F1-Score :  0.7844983446283278
CV No Top-up Service F1-Score :  0.740141478613443


In [ ]:
X_train_stacked=pd.concat([X_train,y_pred_train],axis=1)
X_cv_stacked=pd.concat([X_cv,y_pred_cv],axis=1)
X_test_stacked=pd.concat([X_test,y_pred_test],axis=1)

In [ ]:
clf_svm_stacked=OneVsRestClassifier(RandomForestClassifier(n_estimators=150, max_depth=11,random_state=42,class_weight='balanced'))
clf_svm_stacked.fit(X_train_stacked,y_train)

OneVsRestClassifier(estimator=RandomForestClassifier(bootstrap=True,
                                                     ccp_alpha=0.0,
                                                     class_weight='balanced',
                                                     criterion='gini',
                                                     max_depth=11,
                                                     max_features='auto',
                                                     max_leaf_nodes=None,
                                                     max_samples=None,
                                                     min_impurity_decrease=0.0,
                                                     min_impurity_split=None,
                                                     min_samples_leaf=1,
                                                     min_samples_split=2,
                                                     min_weight_fraction_leaf=0.0,
                                        

In [ ]:
y_pred=clf_svm_stacked.predict(X_train_stacked)
F1_score=f1_score(y_train,y_pred,average='macro')
print('F1-Score : ',F1_score)

F1-Score :  0.4858589446447998


In [ ]:
y_pred=clf_svm_stacked.predict(X_cv_stacked)
F1_score=f1_score(y_cv,y_pred,average='macro')
print('F1-Score : ',F1_score)

F1-Score :  0.3665209207434677


In [ ]:
y_pred_test=pd.DataFrame(clf_svm_stacked.predict(X_test_stacked),columns=y_train.columns)

In [ ]:
y_test_labels=np.argmax(y_pred_test.values,axis=1)
top_up_test=[]
for i in y_test_labels:
  top_up_test.append(labels[i])
test_ids=test_data['ID']
predictions=pd.DataFrame(columns=['ID','Top-up Month'])
predictions['ID']=test_ids
predictions['Top-up Month']=top_up_test
predictions.to_csv('FinalSubmission.csv',index=False)

In [ ]:
predictions.head()

,ID,Top-up Month
0,4,No Top-up Service
1,5,> 48 Months
2,6,No Top-up Service
3,25,No Top-up Service
4,119,No Top-up Service
